# Extraction symbolique de F(α) et G(α)

Ce notebook lit les résultats de calibration `02_As_ns_vs_alpha.csv`, ajuste un polynôme en α pour F(α) et G(α), reconstruit les expressions symboliques, et exporte la table des coefficients.

In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd
import sympy as sp

# 1️⃣ Définition des chemins
ROOT = Path.cwd().parents[1]  # si lancé depuis zz-scripts/chapter02
DATA_IN = ROOT / "zz-data" / "chapter02" / "02_As_ns_vs_alpha.csv"
DATA_OUT = ROOT / "zz-data" / "chapter02"
DATA_OUT.mkdir(parents=True, exist_ok=True)

In [ ]:
# 2️⃣ Lecture des données et calcul des séries
df = pd.read_csv(DATA_IN)
alpha = df["alpha"].values
As = df["A_s"].values
ns = df["n_s"].values

# Constantes Planck 2018
A_s0 = 2.10e-9
ns0 = 0.9649

# 3️⃣ Séries F(α)-1 et G(α)
Fm1 = As / A_s0 - 1
Gm = ns - ns0

# 4️⃣ Ajustement polynômial (ordre 2)
ordre = 2
coef_F = np.polyfit(alpha, Fm1, ordre)  # [c2, c1, c0]
coef_G = np.polyfit(alpha, Gm, ordre)  # [d2, d1, d0]

# 5️⃣ Reconstruction symbolique
a = sp.symbols("alpha")
F_expr = 1 + sum(coef_F[i] * a ** (ordre - i) for i in range(ordre + 1))
G_expr = sum(coef_G[i] * a ** (ordre - i) for i in range(ordre + 1))

print("Expression symbolique pour F(α) :")
sp.pprint(sp.simplify(F_expr))
print("\nExpression symbolique pour G(α) :")
sp.pprint(sp.simplify(G_expr))

In [ ]:
# 6️⃣ Export des coefficients vers CSV
rows = []
for i, c in enumerate(coef_F[::-1]):  # c0 → c2
    rows.append({"fonc": "F", "ordre": i, "coeff": float(c)})
for i, d in enumerate(coef_G[::-1]):  # d0 → d2
    rows.append({"fonc": "G", "ordre": i, "coeff": float(d)})

df_out = pd.DataFrame(rows)
OUT_CSV = DATA_OUT / "02_FG_series.csv"
df_out.to_csv(OUT_CSV, index=False, float_format="%.6e")
print(f"→ Séries F/G exportées dans {OUT_CSV}")

In [ ]:
# 7️⃣ Tests/Assertions
assert set(df_out[df_out.fonc == "F"].ordre) == {0, 1, 2}, "Termes F manquants"
assert set(df_out[df_out.fonc == "G"].ordre) == {0, 1, 2}, "Termes G manquants"
print("✅ Tous les termes F et G jusqu'à l'ordre 2 sont présents.")